In [ ]:
# Code made by Teo Lombardo - JLU University (Giessen, Germany) - OrcID: https://orcid.org/0000-0001-7950-9077

In [ ]:
# Data manipulation
from os import path
import numpy as np
import pandas as pd
from itertools import groupby
import os
import csv
import copy
import datetime

# Plotting
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mplimg
from matplotlib.colors import ListedColormap
import seaborn as sns

# Images manipulation
from skimage import img_as_float, img_as_ubyte, exposure, feature, morphology, measure, segmentation, transform, color
from skimage.restoration import denoise_nl_means, unsupervised_wiener, denoise_tv_bregman, denoise_bilateral
from skimage.util import compare_images, invert
from skimage.filters import threshold_multiotsu, difference_of_gaussians, gaussian, unsharp_mask
from skimage.morphology import disk
from skimage.registration import phase_cross_correlation
from skimage.io import imread, imsave
from scipy import stats, misc, ndimage
import cv2
import imutils
import tifffile

In [ ]:
# Import the images of interest: 

POI = ["NMC signals", "Carbons", "Carbonates.tx", "CC"] # Input
int_phases = copy.copy(POI)
int_phases.remove("CC")
POI.append("total")
mains = ["NMC signals", "Carbons"] # Input

path = r"C:\Users\teolo\Documents\Teo\Papers\SIMS for microstructure characterization\Manuscript\Possible images\NMC 9014 analyses\Test analysis"
measurements = os.listdir(path)

today = datetime.datetime.now()
date_time = today.strftime("%Y_%m_%d")
#print("date and time:",date_time)

directory_res = path + "\\Results_" + str(date_time)
try: 
    os.makedirs(directory_res)
except:
    pass

dict_all = {}
for m in measurements:
    path_img = path + "\\" + m
    for p in POI:
        if p in m and ".txt" in m:
            print(m)
            dict_all[p] = {}
            with open(path_img, 'r') as f:
                lines = f.readlines()
                f.close() 
                
            count_line = 0
            for line in lines:
                splitted_line = line.split(" ")
                #print(splitted_line)
                if splitted_line[0] != "#":
                    header = count_line+1
                    break
                count_line+=1
            
            max_pixel = 0
            max_I = 0
            Is = []
            for line in lines[header:]:
                splitted_line = line.split(" ")
                pixel = float(splitted_line[0])
                if pixel == 0:
                    max_pixel += 1
                I = float(splitted_line[2])
                if I > max_I:
                    max_I = I
                if I > 0:
                    Is.append(I)

            median_n = np.median(Is) # Tried to use it but probably not good idea      
            dict_all[p]["I"] = np.zeros((max_pixel, max_pixel))
            dict_I = dict_all[p]["I"]
            dict_all[p]["In"] = np.zeros((max_pixel, max_pixel)) 
            dict_In = dict_all[p]["In"]
            for line in lines[header:]:
                splitted_line = line.split(" ")
                X = int(splitted_line[0]) 
                Y = int(splitted_line[1]) 
                I = float(splitted_line[2])
                In = I/max_I
                    
                dict_I[Y][X] = I
                dict_In[Y][X] = In
                           
# Report Intensity-based images as 8-bit images
filt_size = 3
dict_imgs_cc = {}
dict_imgs_cc_filtered = {}
dict_imgs = {}
dict_imgs_filtered = {}
dict_imgs_mains = {}
dict_imgs_mains_filtered = {}
for p in POI:
    dict_imgs_cc[p] = dict_all[p]["In"]
    dict_imgs_cc_filtered[p] = cv2.medianBlur(img_as_ubyte(dict_imgs_cc[p]), filt_size)
    if p in int_phases:
        dict_imgs[p] = dict_all[p]["In"]
        dict_imgs_filtered[p] = cv2.medianBlur(img_as_ubyte(dict_imgs_cc[p]), filt_size)
    if p in mains:
        dict_imgs_mains[p] = dict_all[p]["In"]
        dict_imgs_mains_filtered[p] = cv2.medianBlur(img_as_ubyte(dict_imgs_cc[p]), filt_size)

In [ ]:
# Calculate image resolution
FoV = 250 # um # Input
number_pixels = dict_imgs_cc["total"].shape[0]
resolution = (FoV / number_pixels) * 1000 # nm
print(resolution)

path_res = directory_res + "\\Resolution.txt"
res_write = open(path_res, 'w')
res_write.write("Resolution (nm/pixel): " + str(resolution))
res_write.close()

In [ ]:
def combining_imgs(dict_imgs, n_channels, skip):
    img_original_nr = np.zeros((max_pixel, max_pixel, n_channels))

    channel_c = 0
    for img_i in dict_imgs:
        if img_i != skip:
            img_c = dict_imgs[img_i]
            yc = 0
            for y in img_c:
                xc = 0
                for x in y:
                    img_original_nr[yc][xc][channel_c] = x
                    xc+=1
                yc+=1  
        channel_c+=1
        
    return img_original_nr

In [ ]:
# Creating the combined image

# All
channels = len(dict_imgs_cc)-1
img_cc_nr = combining_imgs(dict_imgs_cc, channels, "total")
img_cc_filtered_nr = combining_imgs(dict_imgs_cc_filtered, channels, "total")

# Phases of interest
channels = 3 # To have rgb anyway - instad of len(dict_imgs_mains)
img_original_nr = combining_imgs(dict_imgs, channels, "total")
img_filtered_nr = combining_imgs(dict_imgs_filtered, channels, "total")

# Mains
channels = 3 # To have rgb anyway - instad of len(dict_imgs_mains)
img_original_mains_nr = combining_imgs(dict_imgs_mains, channels, "total")
img_filtered_mains_nr = combining_imgs(dict_imgs_mains_filtered, channels, "total")

In [ ]:
# Remove cc
threshold_cc = 5 # Input

fig = plt.figure(figsize=(20,8))
ax1 = fig.add_subplot(1,2,1)
ax1.imshow(img_original_nr)
ax1.title.set_text("Original - with CC")

count_y = 0
for y in img_cc_nr:
    count_x = 0
    for x in y:
        if x[-1] > threshold_cc:
            for chan in range(len(img_original_nr[count_y][count_x])):
                img_original_nr[count_y][count_x][chan] = 0
            for chan in range(len(img_original_mains_nr[count_y][count_x])):
                img_original_mains_nr[count_y][count_x][chan] = 0
        count_x+=1
    count_y+=1

ax2 = fig.add_subplot(1,2,2)
ax2.imshow(img_original_nr)
ax2.title.set_text("Original - without CC")

In [ ]:
# Lightness enhancment
def light_enh(img, channels, factor_l):
    if channels == 3:
        img_light = np.zeros((img.shape[0], img.shape[1], img.shape[2]))

        yc = 0
        for y in img:
            xc = 0
            for x in y:
                cc = 0
                for c in x:
                    if c > 0:
                        img_light[yc][xc][cc] = c + factor_l
                    cc+=1
                xc+=1
            yc+=1
            
    if channels == 2:
        img_light = np.zeros((img.shape[0], img.shape[1]))
        
        yc = 0
        for y in img:
            xc = 0
            for x in y:
                if x > 0:
                    img_light[yc][xc] = x + factor_l
                xc+=1
            yc+=1
    
    #fig = plt.figure(figsize=(7,7))
    #plt.imshow(img_light)
    return img_light

In [ ]:
# Rotate image 
factor_l = 0 # Input (disabled at the moment)

#if factor_l > 0:
#    img_original_nr_l = light_enh(img_original_nr, channels, factor_l)
#else:
#    img_original_nr_l = img_original_nr

angle_r = 185 # Inputs
path_angle = directory_res + "\\Rotation angle and lightening.txt"
ang_write = open(path_angle, 'w')
ang_write.write("Angle rotation (degree): " + str(angle_r) + "\n")
ang_write.write("Factor lightening: " + str(factor_l))
ang_write.close()

img_cc = imutils.rotate(img_cc_nr, angle=angle_r)
img_cc_filtered = imutils.rotate(img_cc_filtered_nr, angle=angle_r)
img_original = imutils.rotate(img_original_nr, angle=angle_r)
img_filtered = imutils.rotate(img_filtered_nr, angle=angle_r)
img_original_mains = imutils.rotate(img_original_mains_nr, angle=angle_r)
img_filtered_mains = imutils.rotate(img_filtered_mains_nr, angle=angle_r)
#if factor_l > 0:
#    img_original_l = light_enh(img_original, channels, factor_l)
#else:
#    img_original_l = img_original

fig = plt.figure(figsize=(20,8))
ax1 = fig.add_subplot(1,2,1)
ax1.imshow(img_original_nr)
ax1.title.set_text("Original - not rotated")
ax2 = fig.add_subplot(1,2,2)
ax2.imshow(img_original)
ax2.title.set_text("Original - rotated")

path_save = directory_res + "\\Images rotation.png"
plt.savefig(path_save, dpi=600)

fig = plt.figure(figsize=(20,8))
ax1 = fig.add_subplot(1,2,1)
ax1.imshow(img_original_mains_nr)
ax1.title.set_text("Original - not rotated")
ax2 = fig.add_subplot(1,2,2)
ax2.imshow(img_original_mains)
ax2.title.set_text("Original - rotated")

path_save = directory_res + "\\Images rotation - mains.png"
plt.savefig(path_save, dpi=600)

In [ ]:
# Find or manually define limits for image cropping

manual_cut = 1 # Input

if manual_cut == 0:
    thresold_pixel1 = [0.1,0.1,0.1]
    for count_y in range(len(img_original)):
        ref_tester1 = any([img_original[count_y][1][i]>thresold_pixel1[i] for i in range(len(thresold_pixel1))])
        ref_tester2 = any([img_original[count_y][100][i]>thresold_pixel1[i] for i in range(len(thresold_pixel1))])
        ref_tester3 = any([img_original[count_y][200][i]>thresold_pixel1[i] for i in range(len(thresold_pixel1))])
        ref_tester4 = any([img_original[count_y][350][i]>thresold_pixel1[i] for i in range(len(thresold_pixel1))])
        if ref_tester1 == True or ref_tester2 == True or ref_tester3 == True or ref_tester4 == True:
            limit_up = count_y
            print("up found")
            break

    thresold_pixel2 = [0,0,0]
    for count_y in range(len(img_original)):
        ref_tester1 = all([img_original[-count_y][1][i]>thresold_pixel2[i] for i in range(len(thresold_pixel2))])
        ref_tester2 = all([img_original[-count_y][100][i]>thresold_pixel2[i] for i in range(len(thresold_pixel2))])
        ref_tester3 = all([img_original[-count_y][200][i]>thresold_pixel2[i] for i in range(len(thresold_pixel2))])
        ref_tester4 = all([img_original[-count_y][350][i]>thresold_pixel2[i] for i in range(len(thresold_pixel2))])
        if ref_tester1 == True or ref_tester2 == True or ref_tester3 == True or ref_tester4 == True:
            limit_down = img_original.shape[0] - count_y
            print("bottom found")
            break

if manual_cut == 1:
    limit_up = 850 # Input
    limit_down = 1200 # Input
    
print(limit_up, limit_down)
path_angle = directory_res + "\\Cutting limits.txt"
ang_write = open(path_angle, 'w')
ang_write.write("Limits up and down = " + str(limit_up) + " and "+ str(limit_down))
ang_write.close()

img_cropped = img_original[limit_up:limit_down, :]
img_filtered_cropped = img_filtered[limit_up:limit_down, :]
img_cropped_mains = img_original_mains[limit_up:limit_down, :]
img_filtered_mains_cropped = img_filtered_mains[limit_up:limit_down, :]
img_total = dict_imgs_cc["total"][limit_up:limit_down, :]

if factor_l > 0:
    img_cropped_l = light_enh(img_cropped, channels, factor_l)
else:
    img_cropped_l = img_cropped
plt.imshow(img_cropped_l)

In [ ]:
# Recovering the associated SE image (generated through the primary ion gun) - if any (otherrwise you can skip this part, as it is optional)
folder_name = path
for m in measurements:
    if ".bmp" in m:
        file_name_SE = m
        print(m)
        break
path_SE = folder_name + "\\" + file_name_SE
img_SE = img_as_ubyte(imread((path_SE), as_gray=True))
img_SE_rot = imutils.rotate(img_SE, angle=angle_r)
factor_rescale = img_SE.shape[0]/img_original.shape[0]
limit_up_SE = int(limit_up*factor_rescale)
limit_down_SE = int(limit_down*factor_rescale)
img_SE_cropped = img_SE_rot[limit_up_SE:limit_down_SE, :]



fig = plt.figure(figsize=(20,8))
ax1 = fig.add_subplot(1,2,1)
ax1.imshow(img_SE_rot, cmap="gray")
ax1.title.set_text("Original SE rotated")
ax2 = fig.add_subplot(1,2,2)
ax2.imshow(img_SE_cropped, cmap="gray")
ax2.title.set_text("Cropped and rotatedSE")

In [ ]:
# Define single channel images
# Original image
img_red = np.zeros((img_cropped.shape[0], img_cropped.shape[1]))
img_green = np.zeros((img_cropped.shape[0], img_cropped.shape[1]))
img_blue = np.zeros((img_cropped.shape[0], img_cropped.shape[1]))

yc = 0
for y in img_cropped:
    xc = 0
    for x in y:
        img_red[yc][xc] = x[0]
        img_green[yc][xc] = x[1]
        img_blue[yc][xc] = x[2]
        xc+=1
    yc+=1    
    
# Filtered image
img_filtered_red = np.zeros((img_filtered_cropped.shape[0], img_filtered_cropped.shape[1]))
img_filtered_green = np.zeros((img_filtered_cropped.shape[0], img_filtered_cropped.shape[1]))
img_filtered_blue = np.zeros((img_filtered_cropped.shape[0], img_filtered_cropped.shape[1]))

yc = 0
for y in img_filtered_cropped:
    xc = 0
    for x in y:
        img_filtered_red[yc][xc] = x[0]
        img_filtered_green[yc][xc] = x[1]
        img_filtered_blue[yc][xc] = x[2]
        xc+=1
    yc+=1   

    
red_name = POI[0]
green_name = POI[1]
bue_name = POI[2]

# Plotting single channel images
fig = plt.figure(figsize=(20,8))
ax1 = fig.add_subplot(1,3,1)
ax1.imshow(img_red, cmap="hot")
ax1.title.set_text(red_name)
ax2 = fig.add_subplot(1,3,2)
ax2.imshow(img_green, cmap="hot")
ax2.title.set_text(green_name)
ax3 = fig.add_subplot(1,3,3)
ax3.imshow(img_blue, cmap="hot")
ax3.title.set_text(bue_name)

path_save = directory_res + "\\Single channel images.png"
plt.savefig(path_save, dpi=600)

In [ ]:
def rgb_segmenting(img_cropped, blue_splitting):
    segmented_image = np.zeros((img_cropped.shape[0], img_cropped.shape[1], img_cropped.shape[2]))
    
    limits_blue = {}
    for s in range(blue_splitting):
        limits_blue[s] = blues_tert.categories[s]

    for y in range(segmented_image.shape[0]):
        for x in range(segmented_image.shape[1]):
            red_pixel = img_cropped[y][x][0]
            green_pixel = img_cropped[y][x][1]
            blue_pixel = img_cropped[y][x][2]
            if red_pixel > thresold_seg and red_pixel > green_pixel:
                segmented_image[y][x] = [255, 0, 0]
            if green_pixel > thresold_seg and green_pixel > red_pixel*factor_advantage_AM:
                segmented_image[y][x] = [0, 255, 0]

            if blue_pixel > thresold_seg:
                if red_pixel < thresold_seg and green_pixel < thresold_seg:
                    segmented_image[y][x] = [0, 0, 255]

                if red_pixel > thresold_seg and red_pixel > green_pixel:
                    if blue_splitting > 1:
                        for s in range(blue_splitting):
                            if s == 0:
                                if blue_pixel in limits_blue[s]:
                                    segmented_image[y][x] = [255, 0, 125]
                            if s == 1:
                                if blue_pixel in limits_blue[s]:
                                    segmented_image[y][x] = [255, 0, 255]
                            if s == 2:
                                if blue_pixel in limits_blue[s]:
                                    segmented_image[y][x] = [125, 0, 255]
                    if blue_splitting == 1:
                        if blue_pixel > thresold_seg:
                            segmented_image[y][x] = [255, 0, 255]

                if green_pixel > thresold_seg and green_pixel > red_pixel*factor_advantage_AM:
                    if blue_splitting > 1:
                        for s in range(blue_splitting):
                            if s == 0:
                                if blue_pixel in limits_blue[s]:
                                    segmented_image[y][x] = [0, 255, 125]
                            if s == 1:
                                if blue_pixel in limits_blue[s]:
                                    segmented_image[y][x] = [0, 255, 255]
                            if s == 2:
                                if blue_pixel in limits_blue[s]:
                                    segmented_image[y][x] = [0, 125, 255]
                    if blue_splitting == 1:
                        if blue_pixel > thresold_seg:
                            segmented_image[y][x] = [0, 255, 255]
                    
    return segmented_image

In [ ]:
# Combining and getting segmented image
img_cropped = img_as_ubyte(img_cropped)
img_cropped_mains = img_as_ubyte(img_cropped_mains)

thresold_seg = 20 # Input

factor_advantage_AM = 1.3 # Input

path_fact_AM = directory_res + "\\AM factor over segmentation.txt"
fct_write = open(path_fact_AM, 'w')
fct_write.write("AM factor over segmentation (advantage if > 1 and vice versa): " + str(factor_advantage_AM))
fct_write.close()

blues_pixels = []

for y in img_cropped:
    for x in y:
        if x[2] > thresold_seg:
            blues_pixels.append(x[2])  
        
blues_pixels = np.array(blues_pixels)
blue_splitting = 1
blues_tert = pd.qcut(blues_pixels, blue_splitting)

# Original
segmented_image = rgb_segmenting(img_cropped, blue_splitting)
segmented_image_mains = rgb_segmenting(img_cropped_mains, blue_splitting)

# Filtered image
segmented_filtered_image = rgb_segmenting(img_filtered_cropped, blue_splitting)
segmented_filtered_image_mains = rgb_segmenting(img_filtered_mains_cropped, blue_splitting) 
            
# Visualized segmented and cropped image
fig = plt.figure(figsize=(20,8))
ax1 = fig.add_subplot(1,2,1)
ax1.imshow(img_cropped_l)
ax1.title.set_text("Cropped image")
ax1 = fig.add_subplot(1,2,2)
ax1.imshow(segmented_image)
ax1.title.set_text("Segmented image")

path_save = directory_res + "\\Original cropped-rotated and segmented image.png"
plt.savefig(path_save, dpi=600)

# Visualized segmented and cropped image - mains
fig = plt.figure(figsize=(20,8))
ax1 = fig.add_subplot(1,2,1)
ax1.imshow(img_cropped_mains)
ax1.title.set_text("Cropped image - mains")
ax1 = fig.add_subplot(1,2,2)
ax1.imshow(segmented_image_mains)
ax1.title.set_text("Segmented image - mains")

path_save = directory_res + "\\Original cropped-rotated and segmented image - mains.png"
plt.savefig(path_save, dpi=600)

# Visualized filtered images
fig = plt.figure(figsize=(20,8))
ax1 = fig.add_subplot(1,2,1)
ax1.imshow(segmented_filtered_image)
ax1.title.set_text("Filtered - all")
ax1 = fig.add_subplot(1,2,2)
ax1.imshow(segmented_filtered_image_mains)
ax1.title.set_text("Filtered - mains")

path_save = directory_res + "\Segmented filtered images - comparison.png"
plt.savefig(path_save, dpi=600)

In [ ]:
# Measure percentages in volume - on images "all"
# Segmented
print("Segmented")
count_red = 0
count_green = 0
count_other=0
for y in segmented_image:
    for x in y:
        if x[0]>0:
            count_red+=1
        if x[1]>0:
            count_green+=1
        if x[0] == 0 and x[1] == 0:
            count_other+=1

# Calculating percentages
perc_red = count_red/(segmented_image.shape[0]*segmented_image.shape[1])
perc_green = count_green/(segmented_image.shape[0]*segmented_image.shape[1])
perc_others = count_other/(segmented_image.shape[0]*segmented_image.shape[1])

print("Overall percentages:")
print("Red: ", perc_red)
print("Green: ", perc_green)
print("Others: ", perc_others)
print("")

red_green_perc_rel = count_red/(count_red+count_green)
green_red_perc_rel = count_green/(count_red+count_green)
print("Relative percentages:")
print("Red: ", red_green_perc_rel)
print("Green: ", green_red_perc_rel)

# Print in csv file
Material = ["Carbon (overall)", "Si (overall)", "Others (overall)", "Carbon (relative)", "Si (relative)"]
Percentage = [perc_red, perc_green, perc_others, red_green_perc_rel, green_red_perc_rel]

df_int_perc = pd.DataFrame(list(zip(Material, Percentage)),
               columns =['Material', "Volume fraction"])

path_save = directory_res + "\\Volume fractions segmented.csv"
df_int_perc.to_csv(path_save, index=False)

print("")
print("")

# Filtered  Segmented
print("Filtered Segmented")
count_red = 0
count_green = 0
count_other=0
for y in segmented_filtered_image:
    for x in y:
        if x[0]>0:
            count_red+=1
        if x[1]>0:
            count_green+=1
        if x[0] == 0 and x[1] == 0:
            count_other+=1

# Calculating percentages
perc_red = count_red/(segmented_filtered_image.shape[0]*segmented_filtered_image.shape[1])
perc_green = count_green/(segmented_filtered_image.shape[0]*segmented_filtered_image.shape[1])
perc_others = count_other/(segmented_filtered_image.shape[0]*segmented_filtered_image.shape[1])

print("Overall percentages:")
print("Red: ", perc_red)
print("Green: ", perc_green)
print("Others: ", perc_others)
print("")

red_green_perc_rel = count_red/(count_red+count_green)
green_red_perc_rel = count_green/(count_red+count_green)
print("Relative percentages:")
print("Red: ", red_green_perc_rel)
print("Green: ", green_red_perc_rel)

# Print in csv file
Material = ["Carbon (overall)", "Si (overall)", "Others (overall)", "Carbon (relative)", "Si (relative)"]
Percentage = [perc_red, perc_green, perc_others, red_green_perc_rel, green_red_perc_rel]

df_int_perc = pd.DataFrame(list(zip(Material, Percentage)),
               columns =['Material', "Volume fraction"])

path_save = directory_res + "\\Volume fractions filtered segmented.csv"
df_int_perc.to_csv(path_save, index=False)

In [ ]:
# Checking segmented and filtered image with respect to SE image 
fig = plt.figure(figsize=(20,8))
ax1 = fig.add_subplot(1,3,1)
ax1.imshow(img_SE_rot, cmap="gray")
ax1.title.set_text("Original SE rotated")
ax2 = fig.add_subplot(1,3,2)
ax2.imshow(img_SE_cropped, cmap="gray")
ax2.title.set_text("Cropped and rotatedSE")
ax3 = fig.add_subplot(1,3,3)
ax3.imshow(segmented_filtered_image)
ax3.title.set_text("Segmented and filtered SI")

path_save = directory_res + "\\SE cropping and segmented image.png"
plt.savefig(path_save, dpi=600)

In [ ]:
def segmenting(segmented_image, blue_splitting):
    prop_segmented = np.zeros((segmented_image.shape[0], segmented_image.shape[1]))
    for y in range(segmented_image.shape[0]):
        for x in range(segmented_image.shape[1]):
            if segmented_image[y][x][0] == 255: 
                if segmented_image[y][x][2] == 0 : # Carbon
                    prop_segmented[y][x] = 1 
                if blue_splitting > 1:
                    if segmented_image[y][x][2] == 125 : # Carbon + low degradation products
                        prop_segmented[y][x] = 4 
                    if segmented_image[y][x][2] == 255 : # Carbon + medium degradation products
                        prop_segmented[y][x] = 5
            if blue_splitting > 1:
                if segmented_image[y][x][0] == 125: # Carbon + high degradation products
                        prop_segmented[y][x] = 6
            if blue_splitting == 1:
                if segmented_image[y][x][0] == 255 and segmented_image[y][x][2] == 255: # Carbon + high degradation products
                        prop_segmented[y][x] = 4

                        
            if segmented_image[y][x][1] == 255:
                if segmented_image[y][x][2] == 0 : # Silicon
                    prop_segmented[y][x] = 2
                if blue_splitting > 1:
                    if segmented_image[y][x][2] == 125 : # Silicon + low degradation products
                        prop_segmented[y][x] = 7 
                    if segmented_image[y][x][2] == 255 : # Silicon + medium degradation products
                        prop_segmented[y][x] = 8
            if blue_splitting > 1:
                if segmented_image[y][x][1] == 125: # Silicon + high degradation products
                    prop_segmented[y][x] = 9
            if blue_splitting == 1:
                if segmented_image[y][x][1] == 255 and segmented_image[y][x][2] == 255: # Silicon + high degradation products
                    prop_segmented[y][x] = 5

            if segmented_image[y][x][2] == 255 and segmented_image[y][x][0] == 0 and segmented_image[y][x][1] == 0: # Degradation products
                prop_segmented[y][x] = 3
                
    return prop_segmented

In [ ]:
# Proper segmenting and saving

path_save = directory_res + "\\Segmented_rgb_image.png"
imsave(path_save, segmented_image)

path_save = directory_res + "\\Filtered_rgb_image.png"
imsave(path_save, segmented_filtered_image)

path_save = directory_res + "\\Segmented_rgb_image_main.png"
imsave(path_save, segmented_image_mains)

path_save = directory_res + "\\Filtered_rgb_image_main.png"
imsave(path_save, segmented_filtered_image_mains)

# All
prop_segmented = segmenting(segmented_image, blue_splitting)          
path_save = directory_res + "\\Segmented_image.png"
imsave(path_save, prop_segmented)
path_save = directory_res + "\\Segmented_image.tif"
tifffile.imsave(path_save,prop_segmented)  

prop_segmented_filt = segmenting(segmented_filtered_image, blue_splitting)        
path_save = directory_res + "\\Filtered_segmented_image.png"
imsave(path_save, prop_segmented_filt)
path_save = directory_res + "\\Filtered_segmented_image.tif"
tifffile.imsave(path_save,prop_segmented_filt) 

# Mains
prop_segmented_main = segmenting(segmented_image_mains, blue_splitting)          
path_save = directory_res + "\\Segmented_image_mains.png"
imsave(path_save, prop_segmented_main)
path_save = directory_res + "\\Segmented_image_mains.tif"
tifffile.imsave(path_save,prop_segmented_main)  

prop_segmented_filt_mains = segmenting(segmented_filtered_image_mains, blue_splitting)     
path_save = directory_res + "\\Filtered_segmented_image_mains.png"
imsave(path_save, prop_segmented_filt_mains)
path_save = directory_res + "\\Filtered_segmented_image_mains.tif"
tifffile.imsave(path_save,prop_segmented_filt_mains) 

In [ ]:
# Segmenting with degradation products on xy only

filtered_degr_xy =  np.zeros((prop_segmented_filt.shape[0], prop_segmented_filt.shape[1]))

tests = []
lim_t = 2
for t in range(1,lim_t):
    tests.append(-t)
for t in range(1,lim_t):
    tests.append(t)
    
tests0 = []
lim_t0 = 2
for t in range(1,lim_t0):
    tests0.append(-t)
for t in range(1,lim_t0):
    tests0.append(t)

print("tests = ", tests)
print("tests0 = ", tests0)
    
yc = 0
for y in prop_segmented_filt:
    xc = 0
    for x in y:
        if x == 1 or x == 2 or x == 3:
            filtered_degr_xy[yc][xc] = x
        if x == 4:
            for t in tests:
                try:
                    if prop_segmented_filt[yc][xc+t] == 2 or prop_segmented_filt[yc][xc+t] == 5:
                         filtered_degr_xy[yc][xc] = 3
                except:
                    continue
                try:
                    if prop_segmented_filt[yc+t][xc] == 2 or prop_segmented_filt[yc+t][xc] == 5:
                         filtered_degr_xy[yc][xc] = 3
                except:
                    continue
                    
            for t in tests0:
                try:
                    if prop_segmented_filt[yc][xc+t] == 0:
                         filtered_degr_xy[yc][xc] = 3
                except:
                    continue
                try:
                    if prop_segmented_filt[yc+t][xc] == 0:
                         filtered_degr_xy[yc][xc] = 3
                except:
                    continue
            
            
        if x == 5:
            for t in tests:
                try:
                    if prop_segmented_filt[yc][xc+t] == 1 or prop_segmented_filt[yc][xc+t] == 4 or prop_segmented_filt[yc][xc+t] == 0:
                         filtered_degr_xy[yc][xc] = 3
                except:
                    continue
                try:
                    if prop_segmented_filt[yc+t][xc] == 1 or prop_segmented_filt[yc+t][xc] == 4 or prop_segmented_filt[yc+t][xc] == 0:
                         filtered_degr_xy[yc][xc] = 3
                except:
                    continue
                    
            for t in tests0:
                try:
                    if prop_segmented_filt[yc][xc+t] == 0:
                         filtered_degr_xy[yc][xc] = 3
                except:
                    continue
                try:
                    if prop_segmented_filt[yc+t][xc] == 0:
                         filtered_degr_xy[yc][xc] = 3
                except:
                    continue
                    
                    
        if x == 4 and filtered_degr_xy[yc][xc] != 3:
            filtered_degr_xy[yc][xc] = 1
        if x == 5 and filtered_degr_xy[yc][xc] != 3:
            filtered_degr_xy[yc][xc] = 2
        xc+=1
    yc+=1
    
# Degradation_xy_rgb
filtered_degr_xy_rgb =  np.zeros((prop_segmented_filt.shape[0], prop_segmented_filt.shape[1], 3))

yc = 0
for y in filtered_degr_xy:
    xc=0
    for x in y:
        if x == 1:
            filtered_degr_xy_rgb[yc][xc] = [255,0,0]
        if x == 2:
            filtered_degr_xy_rgb[yc][xc] = [0,255,0]
        if x == 3:
            filtered_degr_xy_rgb[yc][xc] = [0,0,255]
        xc+=1    
    yc+=1
    
# Saving 
path_save = directory_res + "\\Filtered_segmented_degr_xy.png"
imsave(path_save, filtered_degr_xy_rgb)
path_save = directory_res + "\\Filtered_segmented_degr_xy.tif"
tifffile.imsave(path_save,filtered_degr_xy)

In [ ]:
# Interfaces determination (segmentation image level - all phases)

red_green_count = 0
red_blue_count = 0
red_others_count = 0 
red_pores_count = 0

green_red_count = 0
green_blue_count= 0
green_others_count = 0
green_pores_count = 0

blue_others_count = 0
blue_pores_count = 0

ref_image = segmented_filtered_image

yc = 0
for y in ref_image:
    xc = 0
    for x in y:
        if x[0] > 0 and x[2] == 0:
            try:
                # Carbon-Si interface
                if ref_image[yc][xc+1][1] == 255:
                    red_green_count+=1
                if xc != 0 :
                    if ref_image[yc][xc-1][1] == 255:
                        red_green_count+=1
                if ref_image[yc+1][xc][1] == 255:
                    red_green_count+=1
                if yc != 0:
                    if ref_image[yc-1][xc][1] == 255:
                        red_green_count+=1
                
                # Carbon-degradation products interface
                if ref_image[yc][xc+1][2] > 0: #  and ref_image[yc][xc+1][0] == 0
                    red_blue_count+=1
                if xc != 0:
                    if ref_image[yc][xc-1][2] > 0:
                        red_blue_count+=1
                if ref_image[yc+1][xc][2] > 0:
                    red_blue_count+=1
                if yc != 0:
                    if ref_image[yc-1][xc][2] > 0:
                        red_blue_count+=1
                
                # Carbon-other/pore interfaces
                if ref_image[yc][xc+1][0] == 0 and ref_image[yc][xc+1][1] == 0 and ref_image[yc][xc+1][2] == 0:
                    if img_total[yc][xc+1] != 0 :
                        red_others_count+=1
                    else:
                        red_pores_count+=1
                if xc != 0:
                    if ref_image[yc][xc-1][0] == 0 and ref_image[yc][xc-1][1] == 0 and ref_image[yc][xc-1][2] == 0:
                        if img_total[yc][xc-1] != 0 :
                            red_others_count+=1
                        else:
                            red_pores_count+=1
                if ref_image[yc+1][xc][0] == 0 and ref_image[yc+1][xc][1] == 0 and ref_image[yc+1][xc][2] == 0:
                    if img_total[yc+1][xc] != 0 :
                        red_others_count+=1
                    else:
                        red_pores_count+=1
                if yc != 0:
                    if ref_image[yc-1][xc][0] == 0 and ref_image[yc-1][xc][1] == 0 and ref_image[yc-1][xc][2] == 0:
                        if img_total[yc-1][xc] != 0 :
                            red_others_count+=1
                        else:
                            red_pores_count+=1
            except:
                continue
                
        if x[0] > 0 and x[2] > 0:
            #red_blue_count+=1 # -- I consider only the interfaces on x/y in this case
            try:
                # Carbon-Si interface
                if ref_image[yc][xc+1][1] == 255:
                    red_green_count+=1
                if xc != 0 :
                    if ref_image[yc][xc-1][1] == 255:
                        red_green_count+=1
                if ref_image[yc+1][xc][1] == 255:
                    red_green_count+=1
                if yc != 0:
                    if ref_image[yc-1][xc][1] == 255:
                        red_green_count+=1
                
                # Carbon-other/pore interfaces
                if ref_image[yc][xc+1][0] == 0 and ref_image[yc][xc+1][1] == 0 and ref_image[yc][xc+1][2] == 0:
                    if img_total[yc][xc+1] != 0 :
                        red_others_count+=1
                    else:
                        red_pores_count+=1
                if xc != 0:
                    if ref_image[yc][xc-1][0] == 0 and ref_image[yc][xc-1][1] == 0 and ref_image[yc][xc-1][2] == 0:
                        if img_total[yc][xc-1] != 0 :
                            red_others_count+=1
                        else:
                            red_pores_count+=1
                if ref_image[yc+1][xc][0] == 0 and ref_image[yc+1][xc][1] == 0 and ref_image[yc+1][xc][2] == 0:
                    if img_total[yc+1][xc] != 0 :
                        red_others_count+=1
                    else:
                        red_pores_count+=1
                if yc != 0:
                    if ref_image[yc-1][xc][0] == 0 and ref_image[yc-1][xc][1] == 0 and ref_image[yc-1][xc][2] == 0:
                        if img_total[yc-1][xc] != 0 :
                            red_others_count+=1
                        else:
                            red_pores_count+=1
            except:
                continue
                
        if x[1] > 0 and x[2] == 0:
            try:
                # Si-Carbon interface
                if ref_image[yc][xc+1][0] == 255:
                    green_red_count+=1
                if xc != 0:
                    if ref_image[yc][xc-1][0] == 255:
                        green_red_count+=1
                if ref_image[yc+1][xc][0] == 255:
                    green_red_count+=1
                if yc != 0:
                    if ref_image[yc-1][xc][0] == 255:
                        green_red_count+=1
                
                # Si-Degradation products interface
                if ref_image[yc][xc+1][2] > 0:
                    green_blue_count+=1
                if xc != 0:
                    if ref_image[yc][xc-1][2] > 0:
                        green_blue_count+=1
                if ref_image[yc+1][xc][2] > 0:
                    green_blue_count+=1
                if yc != 0:
                    if ref_image[yc-1][xc][2] > 0:
                        green_blue_count+=1
                
                # Si-others/pores interfaces
                if ref_image[yc][xc+1][0] == 0 and ref_image[yc][xc+1][1] == 0 and ref_image[yc][xc+1][2] == 0:
                    if img_total[yc][xc+1] != 0 :
                        green_others_count+=1
                    else:
                        green_pores_count
                if xc != 0:
                    if ref_image[yc][xc-1][0] == 0 and ref_image[yc][xc-1][1] == 0 and ref_image[yc][xc-1][2] == 0:
                        if img_total[yc][xc-1] != 0 :
                            green_others_count+=1
                        else:
                            green_pores_count
                if ref_image[yc+1][xc][0] == 0 and ref_image[yc+1][xc][1] == 0 and ref_image[yc+1][xc][2] == 0:
                    if img_total[yc+1][xc] != 0 :
                        green_others_count+=1
                    else:
                        green_pores_count
                if yc != 0:
                    if ref_image[yc-1][xc][0] == 0 and ref_image[yc-1][xc][1] == 0 and ref_image[yc-1][xc][2] == 0:
                        if img_total[yc-1][xc] != 0 :
                            green_others_count+=1
                        else:
                            green_pores_count
            except:
                continue
                
        if x[1] > 0 and x[2] > 0:
            #green_blue_count+=1 # -- I consider only the interfaces on x/y in this case
            try:
                # Si-Carbon interface
                if ref_image[yc][xc+1][0] == 255:
                    green_red_count+=1
                if xc != 0:
                    if ref_image[yc][xc-1][0] == 255:
                        green_red_count+=1
                if ref_image[yc+1][xc][0] == 255:
                    green_red_count+=1
                if yc != 0:
                    if ref_image[yc-1][xc][0] == 255:
                        green_red_count+=1
                
                # Si-others/pores interfaces
                if ref_image[yc][xc+1][0] == 0 and ref_image[yc][xc+1][1] == 0 and ref_image[yc][xc+1][2] == 0:
                    if img_total[yc][xc+1] != 0 :
                        green_others_count+=1
                    else:
                        green_pores_count
                if xc != 0:
                    if ref_image[yc][xc-1][0] == 0 and ref_image[yc][xc-1][1] == 0 and ref_image[yc][xc-1][2] == 0:
                        if img_total[yc][xc-1] != 0 :
                            green_others_count+=1
                        else:
                            green_pores_count
                if ref_image[yc+1][xc][0] == 0 and ref_image[yc+1][xc][1] == 0 and ref_image[yc+1][xc][2] == 0:
                    if img_total[yc+1][xc] != 0 :
                        green_others_count+=1
                    else:
                        green_pores_count
                if yc != 0:
                    if ref_image[yc-1][xc][0] == 0 and ref_image[yc-1][xc][1] == 0 and ref_image[yc-1][xc][2] == 0:
                        if img_total[yc-1][xc] != 0 :
                            green_others_count+=1
                        else:
                            green_pores_count
            except:
                continue
            
        if x[2] >0:
            try:
                if ref_image[yc][xc+1][0] == 0 and ref_image[yc][xc+1][1] == 0 and ref_image[yc][xc+1][2] == 0:
                    if img_total[yc][xc+1] != 0 :
                        blue_others_count+=1
                    else:
                        blue_pores_count
                if xc != 0:
                    if ref_image[yc][xc-1][0] == 0 and ref_image[yc][xc-1][1] == 0 and ref_image[yc][xc-1][2] == 0:
                        if img_total[yc][xc-1] != 0 :
                            blue_others_count+=1
                        else:
                            blue_pores_count
                if ref_image[yc+1][xc][0] == 0 and ref_image[yc+1][xc][1] == 0 and ref_image[yc+1][xc][2] == 0:
                    if img_total[yc+1][xc] != 0 :
                        blue_others_count+=1
                    else:
                        blue_pores_count
                if yc != 0:
                    if ref_image[yc-1][xc][0] == 0 and ref_image[yc-1][xc][1] == 0 and ref_image[yc-1][xc][2] == 0:
                        if img_total[yc-1][xc] != 0 :
                            blue_others_count+=1
                        else:
                            blue_pores_count
            except:
                continue 
        xc+=1
    yc+=1


# Calculating interfaces (um and percentages)
red_green_um = red_green_count*resolution/1000
red_blue_um = red_blue_count*resolution/1000
red_others_um = red_others_count*resolution/1000
green_red_um = green_red_count*resolution/1000
green_blue_um= green_blue_count*resolution/1000
green_others_um = green_others_count*resolution/1000
blue_others_um = blue_others_count*resolution/1000
print("Interfaces size (um): ", red_green_um, red_blue_um, green_blue_um)

print("")

red_green_perc = 100*red_green_count/(red_green_count+red_blue_count+red_others_count+red_pores_count)
red_blue_perc = 100*red_blue_count/(red_green_count+red_blue_count+red_others_count+red_pores_count)
red_others_perc = 100*red_others_count/(red_green_count+red_blue_count+red_others_count+red_pores_count)
red_pores_perc = 100*red_pores_count/(red_green_count+red_blue_count+red_others_count+red_pores_count)
print("red-green(%): ", red_green_perc)
print("red-blue(%): ", red_blue_perc)
print("red-others(%): ", red_others_perc)
print("red-pores(%): ", red_pores_perc)

print("")

green_red_perc = 100*green_red_count/(green_red_count+green_blue_count+green_others_count+green_pores_count)
green_blue_perc = 100*green_blue_count/(green_red_count+green_blue_count+green_others_count+green_pores_count)
green_others_perc = 100*green_others_count/(green_red_count+green_blue_count+green_others_count+green_pores_count)
green_pores_perc = 100*green_pores_count/(green_red_count+green_blue_count+green_others_count+green_pores_count)
print("green-red(%): ", green_red_perc)
print("green-blue(%): ", green_blue_perc)
print("green-others(%): ", green_others_perc)
print("green-pores(%): ", green_pores_perc)

print("")

try:
    blue_red_perc = 100*red_blue_count/(red_blue_count+green_blue_count+blue_others_count+blue_pores_count)
except:
    blue_red_perc = 0
try:
    blue_green_perc = 100*green_blue_count/(red_blue_count+green_blue_count+blue_others_count+blue_pores_count)
except:
    blue_green_perc = 0
try:
    blue_others_perc = 100*blue_others_count/(red_blue_count+green_blue_count+blue_others_count+blue_pores_count)
except:
    blue_others_perc = 0
try:
    blue_pores_perc = 100*blue_pores_count/(red_blue_count+green_blue_count+blue_others_count+blue_pores_count)
except:
    blue_pores_perc = 0
print("blue-red(%): ", blue_red_perc)
print("blue-green(%): ", blue_green_perc)
print("blue-others(%): ", blue_others_perc)
print("blue-pores(%): ", blue_pores_perc)

# Print for later plotting
Material = ["NMC", "NMC", "NMC", "NMC", "C", "C", "C",  "C", "Degr", "Degr", "Degr", "Degr"]
Interface = ["NMC-C", "NMC-Degr", "NMC-pores", "NMC-others", "C-NM", "C-Degr", "C-pores", "C-others", "Degr-NMC", "Degr-C", "Degr-pores", "Degr-others"]
Percentage = [red_green_perc, red_blue_perc, red_pores_perc, red_others_perc, 
              green_red_perc, green_blue_perc, green_pores_perc, green_others_perc,
              blue_red_perc, blue_green_perc, blue_pores_perc, blue_others_perc]

df_int_perc = pd.DataFrame(list(zip(Material, Interface, Percentage)),
               columns =['Material', 'Interface', "Percentage"])

path_save = directory_res + "\\interfaces_perc_overall.csv"
df_int_perc.to_csv(path_save, index=False)

In [ ]:
# Percentage of covered surface
red_fresh = 0
red_covered = 0

green_fresh = 0
green_covered = 0

ref_image = segmented_filtered_image

yc = 0
for y in ref_image:
    xc = 0
    for x in y:
        if x[0] == 255 and x[2] == 0:
            red_fresh+=1
        if x[0] > 0 and x[2] > 0:
            red_covered+=1
            
        if x[1] == 255 and x[2] == 0:
            green_fresh+=1
        if x[1] > 0 and x[2] > 0:
            green_covered+=1
            
red_fresh_perc = 100*red_fresh / (red_fresh+red_covered)
red_covered_perc = 100*red_covered / (red_fresh+red_covered)

green_fresh_perc = 100*green_fresh / (green_fresh+green_covered)
green_covered_perc = 100*green_covered / (green_fresh+green_covered)

# Print for later plotting - Naming to be changed as a function of the (inter)phases that you are looking at
Material = ["NMC", "NMC", "C", "C"]
Interface = ["fresh", "covered", "fresh", "covered"]
Percentage = [red_fresh_perc, red_covered_perc, green_fresh_perc, green_covered_perc]

df_int_perc_coverage = pd.DataFrame(list(zip(Material, Interface, Percentage)),
               columns =['Material', 'Condition', "Percentage"])

path_save = directory_res + "\\interfaces_perc_coverage_overall.csv"
df_int_perc_coverage.to_csv(path_save, index=False)

In [ ]:
# Making single channel images binary
img_red_binary = np.zeros((img_red.shape[0], img_red.shape[1]))

threshold_binary = 0.05 # Input
yc = 0
for y in img_red:
    xc = 0
    for x in y:
        if x > threshold_binary:
            img_red_binary[yc][xc] = 1
        xc+=1
    yc+=1
  

img_green_binary = np.zeros((img_green.shape[0], img_green.shape[1]))
yc = 0
for y in img_green:
    xc = 0
    for x in y:
        if x > threshold_binary:
            img_green_binary[yc][xc] = 1
        xc+=1
    yc+=1
    
img_blue_binary = np.zeros((img_blue.shape[0], img_blue.shape[1]))
yc = 0
for y in img_blue:
    xc = 0
    for x in y:
        if x > threshold_binary:
            img_blue_binary[yc][xc] = 1
        xc+=1
    yc+=1

# Plotting binary single channel images
fig = plt.figure(figsize=(20,8))
ax1 = fig.add_subplot(1,3,1)
ax1.imshow(img_red_binary, cmap="hot")
ax1.title.set_text(red_name)
ax2 = fig.add_subplot(1,3,2)
ax2.imshow(img_green_binary, cmap="hot")
ax2.title.set_text(green_name)
ax3 = fig.add_subplot(1,3,3)
ax3.imshow(img_blue_binary, cmap="hot")
ax3.title.set_text(bue_name)

path_save = directory_res + "\\Single channel binary images.png"
plt.savefig(path_save, dpi=600)

path_thres = directory_res + "\\Thresholds.txt"
thres_write = open(path_thres, 'w')
thres_write.write("Threshold for CC removal: " + str(threshold_cc) + "\n")
thres_write.write("Threshold (combining and segmenting image): " + str(thresold_seg) + "\n")
thres_write.write("Threshold (Binary single channel images): " + str(threshold_binary))
thres_write.close()

In [ ]:
# Making single channel images binary - from filtered
def binary_from_seg_rgb(ref_pixel):
    img_binary = np.zeros((segmented_filtered_image.shape[0], segmented_filtered_image.shape[1]))
    yc = 0
    for y in segmented_filtered_image:
        xc = 0
        for x in y:
            if x[ref_pixel] > 0:
                img_binary[yc][xc] = 1
            xc+=1
        yc+=1
    return img_binary

img_red_filt_binary = binary_from_seg_rgb(0)
img_green_filt_binary = binary_from_seg_rgb(1)
img_blue_filt_binary = binary_from_seg_rgb(2)

# Plotting binary single channel images
fig = plt.figure(figsize=(20,8))
ax1 = fig.add_subplot(1,3,1)
ax1.imshow(img_red_filt_binary, cmap="hot")
ax1.title.set_text(red_name)
ax2 = fig.add_subplot(1,3,2)
ax2.imshow(img_green_filt_binary, cmap="hot")
ax2.title.set_text(green_name)
ax3 = fig.add_subplot(1,3,3)
ax3.imshow(img_blue_filt_binary, cmap="hot")
ax3.title.set_text(bue_name)

path_save = directory_res + "\\Single channel from filtered - binary images.png"
plt.savefig(path_save, dpi=600)

In [ ]:
def object_recognition_prop(phase, img_cropped, small_agglomerates, border,
                           kernel_opening, it_opening,
                           kernel_water, it_opening_sb, thres_factor_dist, name_ref):
    
    # Saving hyperparameters used
    path_par = directory_res + "\\" + name_ref + "_domains_identification_parameters.txt"
    par_write = open(path_par, 'w')
    par_write.write("Kernel_opening: " + str(kernel_opening) + "\n")
    par_write.write("It_opening: " + str(it_opening) + "\n")
    par_write.write("Kernel_water: " + str(kernel_water) + "\n")
    par_write.write("It_opening_sb: " + str(it_opening_sb) + "\n")
    par_write.write("Thres_factor_dist: " + str(thres_factor_dist) + "\n")
    par_write.write("Border: " + str(border) + "\n")
    par_write.write("Small agglomerates: " + str(small_agglomerates))
    par_write.close()
    
    # Cleaning small pixels and removing particles touchng the border
    if small_agglomerates == True:
        phase_opened = cv2.morphologyEx(phase, cv2.MORPH_OPEN, kernel_opening, iterations=it_opening)
    if small_agglomerates == False:
        phase_opened = phase
        
    if border == True:
        phase_opened_cl = segmentation.clear_border(phase_opened)
    if border == False:
        phase_opened_cl = phase_opened

    # Plotting removal small (and) touching borders particles
    fig = plt.figure(figsize=(18,6))
    ax1 = fig.add_subplot(1,3,1)
    ax1.imshow(phase, cmap="hot")
    ax1.title.set_text("segmented phase - original")
    ax2 = fig.add_subplot(1,3,2)
    ax2.imshow(phase_opened, cmap="hot")
    ax2.title.set_text("segmented phase - removal very small agglomerates")
    ax2 = fig.add_subplot(1,3,3)
    ax2.imshow(phase_opened_cl, cmap="hot")
    ax2.title.set_text("segmented phase - removal very small agglomerates and borders")
    
    phase_opened_cl = img_as_ubyte(phase_opened_cl)
    sure_bg = cv2.dilate(phase_opened_cl, kernel_water, iterations=it_opening_sb)
    dist_transform =cv2.distanceTransform(phase_opened_cl, cv2.DIST_L2,5)
    ret2, sure_fg = cv2.threshold(dist_transform, thres_factor_dist*dist_transform.max(),255,0)
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg,sure_fg)
    fig = plt.figure(figsize=(12,8))
    ax1 = fig.add_subplot(2,2,1)
    ax1.imshow(phase_opened_cl, cmap="gray")
    ax1.title.set_text("Original phase")
    ax1 = fig.add_subplot(2,2,2)
    ax1.imshow(sure_bg, cmap="gray")
    ax1.title.set_text("Sure background")
    ax1 = fig.add_subplot(2,2,3)
    ax1.imshow(sure_fg, cmap="gray")
    ax1.title.set_text("Sure foreground")
    ax1 = fig.add_subplot(2,2,4)
    ax1.imshow(unknown, cmap="gray")
    ax1.title.set_text("Unknown")
    
    path_save = directory_res + "\\" + name_ref + "_Background_foreground_watershed.png"
    plt.savefig(path_save, dpi=600)
    
    ret3, markers = cv2.connectedComponents(sure_fg)
    markers = markers+10 # To give the background a value different from 0 (the 0 should be the unknown regions for watershed)
    markers[unknown==255] = 0 # Make 0s the unknown pixels

    markers = cv2.watershed(img_cropped,markers)
    
    # Visualizing identified particles
    fig = plt.figure(figsize=(18,9))
    ax1 = fig.add_subplot(1,3,1)
    ax1.imshow(img_cropped, cmap="gray")
    ax1.title.set_text("Original cropped")
    ax1 = fig.add_subplot(1,3,2)
    ax1.imshow(markers, cmap="hot")
    ax1.title.set_text("Watershed")
    ax1 = fig.add_subplot(1,3,3)
    ax1.imshow(phase, cmap="hot")
    ax1.title.set_text("Phase segmented image")
    
    # Visualizing identified particles
    image_label_overlay_ws = color.label2rgb(markers, image=phase)
    #plt.imshow(image_label_overlay_ws)
    
    img_markers = copy.copy(img_cropped)
    img_markers[markers == -1] = [0,0,255]
    
    markers_cp = copy.copy(markers)
    yc = 0
    for y in markers:
        xc = 0
        for x in y:
            if x == -1:
                markers_cp[yc][xc] = 1
            else:
                markers_cp[yc][xc] = 0
            xc+=1
        yc+=1
    
    fig = plt.figure(figsize=(22,16))
    ax1 = fig.add_subplot(1,2,2)
    ax1.imshow(img_markers)
    ax1.title.set_text("Identified particles on original image")
    ax1 = fig.add_subplot(1,2,1)
    ax1.imshow(markers_cp, cmap="hot")
    ax1.title.set_text("Markes of identified domains")
    path_save = directory_res + "\\" + name_ref + "_Identified domains.png"
    plt.savefig(path_save, dpi=600)
    
    
    # Determine domains properties
    props = measure.regionprops_table(markers,segmented_image,
                                      properties=["label",
                                      "area", "equivalent_diameter_area",
                                      "perimeter", "centroid", "solidity", "orientation",
                                       "axis_minor_length", "axis_major_length"])

    properties = pd.DataFrame(props)

    properties["Percentage area"] = properties["area"]/sum(properties["area"])
    properties["area_perim"] = 2*properties["area"]/(properties["perimeter"]*(0.5*properties["equivalent_diameter_area"]))
    properties["circulary_Wadell"] = (properties["equivalent_diameter_area"]*np.pi) / properties["perimeter"]
    properties["Sphericity"] = (properties["axis_minor_length"]) / properties["axis_major_length"] # Same of ImageJ
    # file:///C:/Users/teolo/Downloads/minerals-09-00768.pdf - 
    #Wadell, H.A. Sphericity and roundness of rock particles. J. Geol. 1933, 41, 310–331. - 
    #Wadell, H.A. Volume, shape and roundness of rock particles. J. Geol. 1932, 40, 443–451.
    properties["aspect ratio"] = properties["axis_major_length"]/properties["axis_minor_length"]
    properties["equivalent_diameter_um"] = properties["equivalent_diameter_area"]*resolution/1000

    properties = properties.iloc[1: , :]
    
    return markers, properties

In [ ]:
# Object recognition
phase = img_red_filt_binary
#gray_phase = img_red_grey_HE
kernel_opening = np.ones((3,3), np.uint8) # Used only if small_agglomerates==True 
it_opening = 1 # Used only if small_agglomerates==True 
kernel_water = np.ones((3,3), np.uint8) # To define the sure background
it_opening_sb = 1 # Used for the sure background
thres_factor_dist = 0.05 # the smaller the more agglomerates it finds - used for the sure foreground
border = False
small_agglomerates = True
name_ref = "NMC"

ref_image_wts = copy.copy(segmented_image_mains.astype("uint8"))

markers_NMC, properties_NMC = object_recognition_prop(phase, ref_image_wts, small_agglomerates, border,
                                                                kernel_opening, it_opening, 
                                                                kernel_water, it_opening_sb, thres_factor_dist, name_ref)

In [ ]:
# Plotting domains' properties

properties = properties_NMC
z_centroids = [((phase.shape[0]-z)*resolution/1000) for z in properties['centroid-0']]
path_save = directory_res + "\\" + name_ref + "_properties.csv"
properties.to_csv(path_save, index=False)

col_hist = "red"
cmap = plt.cm.Reds

colours = properties["equivalent_diameter_um"]/max(properties["equivalent_diameter_um"])
size = colours*400

colours = []
for i in properties["equivalent_diameter_um"]:
    value = max(0.3, i**2/max(properties["equivalent_diameter_um"]**2))
    colours.append(value)
col_p = cmap(colours)

fig_, ax_ = plt.subplots(ncols=2, nrows=1, figsize=(14, 6))
sns.histplot(properties["equivalent_diameter_um"], bins=20, ax=ax_[0], color=col_hist, log_scale=True) # log_scale=(True,True) for log scale on both x and y
ax_[0].set_xlabel("Equivalent diameter (µm)", fontsize=16)
ax_[0].set_ylabel("Counts", fontsize=16)
sns.histplot(properties["aspect ratio"], bins=20, ax=ax_[1], color=col_hist)
ax_[1].set_xlabel("Aspect ratio", fontsize=16)
ax_[1].set_ylabel("Counts", fontsize=16)

path_save = directory_res + "\\" + name_ref + "_PSD.png"
plt.savefig(path_save, dpi=600)

fig_, ax_ = plt.subplots(ncols=2, nrows=3, figsize=(16, 12))

sns.histplot(properties["Sphericity"], bins=20, ax=ax_[2,0], color=col_hist, binrange=(0,1))
ax_[2,0].set_xlabel("Sphericity", fontsize=16)
ax_[2,0].set_ylabel("Counts", fontsize=16)
ax_[2,0].set_xlim([0,1])
ax_[2,1].scatter(properties["equivalent_diameter_um"], properties["Sphericity"], s=size, c=col_p)
ax_[2,1].set_ylim([0,1.05])
ax_[2,1].set_xlabel("Equivalent diameter (µm)", fontsize=16)
ax_[2,1].set_ylabel("Sphericity", fontsize=16)
ax_[2,1] = fig.add_subplot(2,3,2)

sns.histplot(properties["circulary_Wadell"], bins=20, ax=ax_[1,0], color=col_hist, binrange=(0,1))
ax_[1,0].set_xlabel("Wadell circularity", fontsize=16)
ax_[1,0].set_ylabel("Counts", fontsize=16)
#ax_[1,0].set_xlim([0,1])
ax_[1,1].scatter(properties["equivalent_diameter_um"], properties["circulary_Wadell"], s=size, c=col_p)
ax_[1,1].set_ylim([0,1.05])
ax_[1,1].set_xlabel("Equivalent diameter (µm)", fontsize=16)
ax_[1,1].set_ylabel("Wadell circularity", fontsize=16)
ax_[1,1] = fig.add_subplot(2,3,4)

sns.histplot(properties["solidity"], bins=20, ax=ax_[0,0], color=col_hist)
ax_[0,0].set_xlabel("Solidity", fontsize=16)
ax_[0,0].set_ylabel("Counts", fontsize=16)
ax_[0,1].scatter(properties["equivalent_diameter_um"], properties["solidity"], s=size, c=col_p)
ax_[0,1].set_ylim([0,1.05])
ax_[0,1].set_xlabel("Equivalent diameter (µm)", fontsize=16)
ax_[0,1].set_ylabel("Solidity", fontsize=16)
ax_[0,1] = fig.add_subplot(2,3,6)

path_save = directory_res + "\\" + name_ref + "_shape_regularity.png"
plt.savefig(path_save, dpi=600)

In [ ]:
# Plotting domains' properties - electrode thickness

fig_, ax_ = plt.subplots(ncols=2, nrows=3, figsize=(16, 12))

sns.histplot(properties["Sphericity"], bins=20, ax=ax_[2,0], color=col_hist, binrange=(0,1))
ax_[2,0].set_xlabel("Sphericity", fontsize=16)
ax_[2,0].set_ylabel("Counts", fontsize=16)
ax_[2,0].set_xlim([0,1])
ax_[2,1].scatter(z_centroids, properties["Sphericity"], s=size, c=col_p)
ax_[2,1].set_ylim([0,1.05])
ax_[2,1].set_xlabel("Electrode thickness (µm)", fontsize=16)
ax_[2,1].set_ylabel("Sphericity", fontsize=16)
ax_[2,1] = fig.add_subplot(2,3,2)

sns.histplot(properties["circulary_Wadell"], bins=20, ax=ax_[1,0], color=col_hist, binrange=(0,1))
ax_[1,0].set_xlabel("Wadell circularity", fontsize=16)
ax_[1,0].set_ylabel("Counts", fontsize=16)
#ax_[1,0].set_xlim([0,1])
ax_[1,1].scatter(z_centroids, properties["circulary_Wadell"], s=size, c=col_p)
ax_[1,1].set_ylim([0,1.05])
ax_[1,1].set_xlabel("Electrode thickness (µm)", fontsize=16)
ax_[1,1].set_ylabel("Wadell circularity", fontsize=16)
ax_[1,1] = fig.add_subplot(2,3,4)

sns.histplot(properties["solidity"], bins=20, ax=ax_[0,0], color=col_hist)
ax_[0,0].set_xlabel("Solidity", fontsize=16)
ax_[0,0].set_ylabel("Counts", fontsize=16)
ax_[0,1].scatter(z_centroids, properties["solidity"], s=size, c=col_p)
ax_[0,1].set_ylim([0,1.05])
ax_[0,1].set_xlabel("Electrode thickness (µm)", fontsize=16)
ax_[0,1].set_ylabel("Solidity", fontsize=16)
ax_[0,1] = fig.add_subplot(2,3,6)

path_save = directory_res + "\\" + name_ref + "_shape_regularity_thickness.png"
plt.savefig(path_save, dpi=600)

In [ ]:
# Creating dictionaries for storing domains' interfaces information
properties = properties_NMC
markers_cp = copy.copy(markers_NMC)

dict_sp_interf = {}
dict_sp_interf_c = {}
for i in properties["label"]:
    dict_sp_interf[i] = {}
    dict_sp_interf[i]["x+"] = {}
    dict_sp_interf[i]["x-"] = {}
    dict_sp_interf[i]["y+"] = {}
    dict_sp_interf[i]["y-"] = {}
    
    dict_sp_interf[i]["x+"]["xc"] = []
    dict_sp_interf[i]["x+"]["yc"] = []
    dict_sp_interf[i]["x-"]["xc"] = []
    dict_sp_interf[i]["x-"]["yc"] = []
    dict_sp_interf[i]["y+"]["xc"] = []
    dict_sp_interf[i]["y+"]["yc"] = []
    dict_sp_interf[i]["y-"]["xc"] = []
    dict_sp_interf[i]["y-"]["yc"] = []
    
    dict_sp_interf_c[i] = {}
    dict_sp_interf_c[i]["count_t"] = 0
    dict_sp_interf_c[i]["red_green"] = 0
    dict_sp_interf_c[i]["red_blue"] = 0
    dict_sp_interf_c[i]["red_others"] = 0
    
# Identifying coordinates of domains' interfaces

yc = 1
for y in markers_cp[1:-1, 1:-1]:
    xc = 1
    for x in y:
        if x!= 10:
            if x == -1:
                if markers_cp[yc][xc+1] > 10:
                    id_i = markers_cp[yc][xc+1]
                    dict_sp_interf[id_i]["x+"]["xc"].append(xc) 
                    dict_sp_interf[id_i]["x+"]["yc"].append(yc) 
                    dict_sp_interf_c[id_i]["count_t"] +=1
                if markers_cp[yc][xc-1] > 10:
                    id_i = markers_cp[yc][xc-1]
                    dict_sp_interf[id_i]["x-"]["xc"].append(xc) 
                    dict_sp_interf[id_i]["x-"]["yc"].append(yc) 
                    dict_sp_interf_c[id_i]["count_t"] +=1
                if markers_cp[yc+1][xc] > 10:
                    id_i = markers_cp[yc+1][xc]
                    dict_sp_interf[id_i]["y+"]["xc"].append(xc) 
                    dict_sp_interf[id_i]["y+"]["yc"].append(yc) 
                    dict_sp_interf_c[id_i]["count_t"] +=1
                if markers_cp[yc-1][xc] > 10:
                    id_i = markers_cp[yc-1][xc]
                    dict_sp_interf[id_i]["y-"]["xc"].append(xc) 
                    dict_sp_interf[id_i]["y-"]["yc"].append(yc)  
                    dict_sp_interf_c[id_i]["count_t"] +=1
        xc+=1
    yc+=1
    
# Counting interfaces at the domain level 

ref_pixel = [255, 0, 0]
other_pixel = [0,0,0]

int_pixel1 = [0, 255, 0]
int_pixel2 = [0, 0, 255]


ref_image = segmented_filtered_image

len_test = 100
#for ref_image in ref_images:
for i in dict_sp_interf:
    for j in dict_sp_interf[i]:
        if j == "x+":
            for k in range(len(dict_sp_interf[i][j]["xc"])):
                xc = dict_sp_interf[i][j]["xc"][k]
                yc = dict_sp_interf[i][j]["yc"][k]
                for l in range(1,len_test+1):
                    #print(l)
                    try:
                        #ref_tester = all([ref_image[yc][xc+l][i]==ref_pixel[i] for i in range(len(ref_pixel))])
                        tester1 = all([ref_image[yc][xc+l][i]==int_pixel1[i] for i in range(len(int_pixel1))])
                        #tester2 = all([ref_image[yc][xc+l][i]==int_pixel2[i] for i in range(len(int_pixel2))])
                        if tester1 == True:
                            dict_sp_interf_c[i]["red_green"]+=1
                            break
                        elif ref_image[yc][xc+l][2] > 0 and ref_image[yc][xc][2] == 0:
                            dict_sp_interf_c[i]["red_blue"]+=1
                            break
                        elif ref_image[yc][xc+l][2] > 0 and ref_image[yc][xc+l][1] > 0:
                            dict_sp_interf_c[i]["red_blue"]+=1
                            break
                        elif ref_image[yc][xc+l][0] == 0 and ref_image[yc][xc+l][1] == 0 and ref_image[yc][xc+l][2] == 0:
                            dict_sp_interf_c[i]["red_others"]+=1
                            break
                    except:
                        continue
                        
        if j == "x-":
            for k in range(len(dict_sp_interf[i][j]["xc"])):
                xc = dict_sp_interf[i][j]["xc"][k]
                yc = dict_sp_interf[i][j]["yc"][k]
                for l in range(1,len_test+1):
                    try:
                        #ref_tester = all([ref_image[yc][xc-l][i]==ref_pixel[i] for i in range(len(ref_pixel))])
                        tester1 = all([ref_image[yc][xc-l][i]==int_pixel1[i] for i in range(len(int_pixel1))])
                        #tester2 = all([ref_image[yc][xc-l][i]==int_pixel2[i] for i in range(len(int_pixel2))])
                        if tester1 == True:
                            dict_sp_interf_c[i]["red_green"]+=1
                            break
                        elif ref_image[yc][xc-l][2] > 0 and ref_image[yc][xc][2] == 0:
                            dict_sp_interf_c[i]["red_blue"]+=1
                            break
                        elif ref_image[yc][xc-l][2] > 0 and ref_image[yc][xc-l][1] > 0:
                            dict_sp_interf_c[i]["red_blue"]+=1
                            break
                        elif ref_image[yc][xc-l][0] == 0 and ref_image[yc][xc-l][1] == 0 and ref_image[yc][xc-l][2] == 0:
                            dict_sp_interf_c[i]["red_others"]+=1
                            break
                    except:
                        continue
                        
        if j == "y+":
            for k in range(len(dict_sp_interf[i][j]["xc"])):
                xc = dict_sp_interf[i][j]["xc"][k]
                yc = dict_sp_interf[i][j]["yc"][k]
                for l in range(1,len_test+1):
                    try:
                        #ref_tester = all([ref_image[yc+l][xc][i]==ref_pixel[i] for i in range(len(ref_pixel))])
                        tester1 = all([ref_image[yc+l][xc][i]==int_pixel1[i] for i in range(len(int_pixel1))])
                        #tester2 = all([ref_image[yc+l][xc][i]==int_pixel2[i] for i in range(len(int_pixel2))])
                        if tester1 == True:
                            dict_sp_interf_c[i]["red_green"]+=1
                            break
                        elif ref_image[yc+l][xc][2] > 0 and ref_image[yc][xc][2] == 0:
                            dict_sp_interf_c[i]["red_blue"]+=1
                            break
                        elif ref_image[yc+l][xc][2] > 0 and ref_image[yc+l][xc][1] > 0:
                            dict_sp_interf_c[i]["red_blue"]+=1
                            break
                        elif ref_image[yc+l][xc][0] == 0 and ref_image[yc+l][xc][1] == 0 and ref_image[yc+l][xc][2] == 0:
                            dict_sp_interf_c[i]["red_others"]+=1
                            break
                    except:
                        continue
                        
        if j == "y-":
            for k in range(len(dict_sp_interf[i][j]["xc"])):
                xc = dict_sp_interf[i][j]["xc"][k]
                yc = dict_sp_interf[i][j]["yc"][k]
                for l in range(1,len_test+1):
                    try:
                        #ref_tester = all([ref_image[yc-l][xc][i]==ref_pixel[i] for i in range(len(ref_pixel))])
                        tester1 = all([ref_image[yc-l][xc][i]==int_pixel1[i] for i in range(len(int_pixel1))])
                        #tester2 = all([ref_image[yc-l][xc][i]==int_pixel2[i] for i in range(len(int_pixel2))])
                        if tester1 == True:
                            dict_sp_interf_c[i]["red_green"]+=1
                            break
                        elif ref_image[yc-l][xc][2] > 0 and ref_image[yc][xc][2] == 0:
                            dict_sp_interf_c[i]["red_blue"]+=1
                            break
                        elif ref_image[yc-l][xc][2] > 0 and ref_image[yc-l][xc][1] > 0:
                            dict_sp_interf_c[i]["red_blue"]+=1
                            break
                        elif ref_image[yc-l][xc][0] == 0 and ref_image[yc-l][xc][1] == 0 and ref_image[yc-l][xc][2] == 0:
                            dict_sp_interf_c[i]["red_others"]+=1
                            break
                    except:
                        continue
                        
# Putting interfaces data in props dataframe

red_green_dom_c = []
red_blue_dom_c = []
red_others_dom_c = []

red_green_dom_p = []
red_blue_dom_p = []
red_others_dom_p = []

for i in dict_sp_interf_c:
    red_green_dom_c.append(dict_sp_interf_c[i]["red_green"])
    red_blue_dom_c.append(dict_sp_interf_c[i]["red_blue"])
    red_others_dom_c.append(dict_sp_interf_c[i]["red_others"])
    
    sum_counts = dict_sp_interf_c[i]["red_green"]+dict_sp_interf_c[i]["red_blue"]+dict_sp_interf_c[i]["red_others"]
    
    if sum_counts > 0:
        perc_dom_red_green = dict_sp_interf_c[i]["red_green"]/sum_counts
        red_green_dom_p.append(perc_dom_red_green)

        perc_dom_red_blue = dict_sp_interf_c[i]["red_blue"]/sum_counts
        red_blue_dom_p.append(perc_dom_red_blue)

        perc_dom_red_others = dict_sp_interf_c[i]["red_others"]/sum_counts
        red_others_dom_p.append(perc_dom_red_others)
    else:
        red_green_dom_p.append(-1)
        red_blue_dom_p.append(-1)
        red_others_dom_p.append(-1)
    
properties["NMC_C_count"] = red_green_dom_c
properties["NMC_Degr_count"] = red_blue_dom_c
properties["NMC_others_count"] = red_others_dom_c

properties["NMC_C (%)"] = red_green_dom_p
properties["NMC_Degr (%)"] = red_blue_dom_p
properties["NMC_others (%)"] = red_others_dom_p

In [ ]:
# Plotting interfaces as a function of the domains' size

properties = copy.copy(properties_NMC)
properties = properties[properties["NMC_C (%)"] != -1]

cmap = plt.cm.Reds
#cmap2 = plt.cm.Reds
#cmap3 = plt.cm.Blues

colours = properties["equivalent_diameter_um"]/max(properties["equivalent_diameter_um"])
size = colours*1000

colours = []
for j in range(3):
    for i in properties["equivalent_diameter_um"]:
        value = max(0.3, i**2/max(properties["equivalent_diameter_um"]**2))
        colours.append(value)

cols_p = cmap(colours)
#col_p2 = cmap2(colours)
#col_p3 = cmap3(colours)

list_percentages = []
list_interfaces = []

sizes = []


for i in range(3):
    for s in size:
        sizes.append(s)


name_i = "NMC_C (%)"
for i in properties[name_i]:
    list_percentages.append(i*100)
    list_interfaces.append("NMC_CBD")
    
name_i = "NMC_Degr (%)"
for i in properties[name_i]:
    list_percentages.append(i*100)
    list_interfaces.append("NMC_Degr")
    
name_i = "NMC_others (%)"
for i in properties[name_i]:
    list_percentages.append(i*100)
    list_interfaces.append("NMC_others")

fig = plt.figure(figsize=(11,8))    
plt.scatter(list_interfaces, list_percentages, s=sizes, c=cols_p, alpha=0.75)
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)
plt.ylim([-5,105])
plt.ylabel("%", fontsize=36)
path_save = directory_res + "\\" + name_ref + "_domains_interfaces.png"
plt.savefig(path_save, dpi=600)

In [ ]:
# Object recognition - green phase
phase = img_green_filt_binary
#gray_phase = img_green_grey_HE
kernel_opening = np.ones((3,3), np.uint8) # Used only if small_agglomerates==True 
it_opening = 1 # Used only if small_agglomerates==True 
kernel_water = np.ones((3,3), np.uint8) # To define the sure background
it_opening_sb = 2 # Used for the sure background
thres_factor_dist = 0.1 # the smaller the more agglomerates it finds - used for the sure foreground
border = False
small_agglomerates = False
name_ref = "Carbon"

ref_image_wts = copy.copy(segmented_image_mains.astype("uint8"))

markers_C, properties_C = object_recognition_prop(phase, ref_image_wts, small_agglomerates, border,
                                               kernel_opening, it_opening, 
                                               kernel_water, it_opening_sb, thres_factor_dist, name_ref)

In [ ]:
# End of the code